In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')
df.shape[1]
print(df['SalePrice'].describe())
df_clean = df.copy()
miss_col = df_clean.columns[df.isnull().any()].tolist()

df_clean = df.copy()
basement_na_list = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
fireplace_na_liat = ['FireplaceQu']
garage_na_list = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
pool_na_list = ['PoolQC']
fence_na_list = ['Fence']
misfeature_na_list = ['MiscFeature']

for missing in miss_col:
    if missing == 'Alley':
        df_clean['Alley'] = df_clean['Alley'].replace(np.nan,'No alley access')
    elif missing in basement_na_list:
        df_clean.loc[:,basement_na_list] = df_clean.loc[:,basement_na_list].replace(np.nan,'No basement')
    elif missing in fireplace_na_liat:
        df_clean.loc[:,fireplace_na_liat] = df_clean.loc[:,fireplace_na_liat].replace(np.nan,'No fireplace')
    elif missing in garage_na_list:
        df_clean.loc[:,garage_na_list] = df_clean.loc[:,garage_na_list].replace(np.nan,'No garage')
    elif missing in pool_na_list:
        df_clean.loc[:,pool_na_list] = df_clean.loc[:,pool_na_list].replace(np.nan,'No pool')
    elif missing in fence_na_list:
        df_clean.loc[:,fence_na_list] = df_clean.loc[:,fence_na_list].replace(np.nan,'No fence')
    elif missing in misfeature_na_list:
        df_clean.loc[:,misfeature_na_list] = df_clean.loc[:,misfeature_na_list].replace(np.nan,'None')


df_clean.isnull().sum()
miss_col_clean = df_clean.columns[df_clean.isnull().any()].tolist()
print(miss_col_clean)
df_clean.loc[:, miss_col_clean].isnull().sum()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64
['LotFrontage', 'MasVnrType', 'MasVnrArea', 'Electrical', 'GarageYrBlt']


LotFrontage    259
MasVnrType       8
MasVnrArea       8
Electrical       1
GarageYrBlt     81
dtype: int64

In [2]:
def find_index_num_str(df):
    is_string = []
    for index in range(df.shape[1]):
        data = df.iloc[:,index]
        is_string.append(is_string_dtype(data))
    # print(is_string)

    index_numerical = [i for i, istring in enumerate(is_string) if istring == False]
    # print(index_false)
    index_string = [i for i, istring in enumerate(is_string) if istring == True]
    
    return index_numerical, index_string

In [22]:
df_feature = df_clean.loc[:, df_clean.columns != 'SalePrice']
df_feature = df_feature.drop(['Id'], axis=1)
df_target = df_clean['SalePrice']

is_string = []
for index in range(df_feature.shape[1]):
    data = df_feature.iloc[:,index]
    is_string.append(is_string_dtype(data))
# print(is_string)

index_numerical = [i for i, istring in enumerate(is_string) if istring == False]
# print(index_false)
index_string = [i for i, istring in enumerate(is_string) if istring == True]

df_num = df_feature.iloc[:, index_numerical]
df_nom = df_feature.iloc[:, index_string]
# df_nom

df_imp_num = df_num.copy()
mean_imputer = SimpleImputer(strategy='mean')
most_freq_imp = SimpleImputer(strategy='most_frequent')

df_imp_num
df_imp_num['LotFrontage'] = df_imp_num['LotFrontage'].fillna(df_imp_num['LotFrontage'].mean())
df_imp_num['GarageYrBlt'] = most_freq_imp.fit_transform(df_imp_num['GarageYrBlt'].values.reshape(-1,1))
df_imp_num['MasVnrArea'] = df_imp_num['MasVnrArea'].fillna(df_imp_num['MasVnrArea'].mean())

df_imp_nom = df_nom.copy()
for i in range(len(df_imp_nom.columns)):
    df_imp_nom.iloc[:,i] = most_freq_imp.fit_transform(df_imp_nom.iloc[:,i].values.reshape(-1,1))

    
df_imp = pd.concat([df_imp_nom, df_imp_num], axis=1)
print(df_imp.isnull().sum().tolist())
# miss_col = df_imp.columns[df_imp.isnull().any()].tolist()
# print(miss_col)
# df_imp['MasVnrArea']

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_base.py:54: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)


In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score

def cv_decision_tree(x, y, kfold):
    clf = DecisionTreeClassifier()
    k_folds = KFold(n_splits = kfold)
    scores = cross_val_score(clf, x, y, cv = k_folds)
    return scores.mean()

# X_knn_enc
# print('Avr. CV of knn imputation: ',cv_decision_tree(X_knn_enc, y_knn, 5))

In [36]:
df_imp_enc = df_imp.copy()
i_num, i_str = find_index_num_str(df_imp_enc)
df_imp_enc.iloc[:, i_str]
# print(index_string)
for col in i_str:
    labelencoder = LabelEncoder()
    df_imp_enc.iloc[:,col] = labelencoder.fit_transform(df_imp_enc.iloc[:,col])
# print(df_imp_enc.isnull().sum().tolist())
df_imp_enc

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,3,1,1,3,3,0,4,0,5,2,...,548,0,61,0,0,0,0,0,2,2008
1,3,1,1,3,3,0,2,0,24,1,...,460,298,0,0,0,0,0,0,5,2007
2,3,1,1,0,3,0,4,0,5,2,...,608,0,42,0,0,0,0,0,9,2008
3,3,1,1,0,3,0,0,0,6,2,...,642,0,35,272,0,0,0,0,2,2006
4,3,1,1,0,3,0,2,0,15,2,...,836,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,3,1,1,3,3,0,4,0,8,2,...,460,0,40,0,0,0,0,0,8,2007
1456,3,1,1,3,3,0,4,0,14,2,...,500,349,0,0,0,0,0,0,2,2010
1457,3,1,1,3,3,0,4,0,6,2,...,252,0,60,0,0,0,0,2500,5,2010
1458,3,1,1,3,3,0,4,0,12,2,...,240,366,0,112,0,0,0,0,4,2010


In [26]:
print('Avr. CV of simple imputation: ',cv_decision_tree(df_imp_enc, df_target, 10))

Avr. CV of simple imputation:  0.007534246575342465


In [37]:
df_simple_imp = pd.concat([df_imp_enc, df_clean['SalePrice']], axis=1)
df_simple_imp.to_csv('df_simple_imp.csv', encoding='utf-8', index=False)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,3,1,1,3,3,0,4,0,5,2,...,0,61,0,0,0,0,0,2,2008,208500
1,3,1,1,3,3,0,2,0,24,1,...,298,0,0,0,0,0,0,5,2007,181500
2,3,1,1,0,3,0,4,0,5,2,...,0,42,0,0,0,0,0,9,2008,223500
3,3,1,1,0,3,0,0,0,6,2,...,0,35,272,0,0,0,0,2,2006,140000
4,3,1,1,0,3,0,2,0,15,2,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,3,1,1,3,3,0,4,0,8,2,...,0,40,0,0,0,0,0,8,2007,175000
1456,3,1,1,3,3,0,4,0,14,2,...,349,0,0,0,0,0,0,2,2010,210000
1457,3,1,1,3,3,0,4,0,6,2,...,0,60,0,0,0,0,2500,5,2010,266500
1458,3,1,1,3,3,0,4,0,12,2,...,366,0,112,0,0,0,0,4,2010,142125
